## Get collected meta data

In [1]:
import pandas as pd

In [2]:
kids_video_game = pd.read_csv('/Users/QiaoLiu1/kids_tube/video_details/video_details_kids_video_game.csv', 
                             index_col = 0)

In [3]:
kids_video_game.head()

,video,title,tags,description,category,caption
0,yTqsHkxuna0,Sky: Children of the Light | Most Relaxing Gam...,"Cryy,cryyfuu,sky: children of the light,sky: c...",I stumbled across this mobile / iOS game title...,Kids video game play,False
1,mYyaScLzieI,Snowboard Kids (N64) Gameplay,"snowboard,kids,n64,nintendo,64,gameplay",Gameplay footage of Snowboard Kids on Nintendo...,Kids video game play,False
2,36seSrWXJOQ,AVENGERS Cartoon Game Videos for Kids - Video ...,"avengers,avengers cartoon,kids videos for kids...",Disney Infinity 3.0 Marvel Battlegrounds video...,Kids video game play,False
3,i22Pp2Th7yA,My Virtual Pet Shop - Fun Kids Game for iPhone...,"IPhone (Mobile Phone),Android (Software),Game,...",Download on the App Store: https://itunes.appl...,Kids video game play,False
4,Am6h1v6ACqQ,지네 장난감 곤충 장난감 숨바꼭질 놀이 Surprise Toy for Kids Ga...,"With Kids,위드키즈,키즈유튜브,예준이,jacob,kids,indoor pla...",Surprise Toy for Kids with papa Funny Video fo...,Kids video game play,False


###  Input stopword, word and 

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import words
stop_words = set(stopwords.words('english'))
stopword_set = set(stopwords.words())
word_set = set(words.words())


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/QiaoLiu1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/QiaoLiu1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/QiaoLiu1/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Prepare matplotlib parameters

In [6]:
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
### set up plotting parameters
from matplotlib import rcParams
plt.style.use('seaborn-poster')
plt.rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Palatino']
rcParams['figure.max_open_warning'] = 30

In [7]:
def remove_stop(text):
    try:
        text_tokens = word_tokenize(text)
        tokens_without_sw = [word for word in text_tokens
                             if (word.lower().strip() not in stopword_set) and (word.lower().strip() in word_set)]
        return tokens_without_sw
    except:
        return ['']

In [ ]:
### tags
### 1. remove number
### 2. lemma
### 2. get bigram for each setence
### 3. get trigram for each sentence